Проверим, корректно ли ICU сортирует? Правда ли тут https://stackoverflow.com/a/1098160 написана?

Зависимости под Windows:

```
> pip install pipwin
> pipwin install pyicu
```

Увы, не всегда работает. Поэтому ставим PyICU вручную отсюда: https://www.lfd.uci.edu/~gohlke/pythonlibs/

* В роли Windows выступала Windows 10
* В роли UNIX-подобной ОС выступал Linux x86\_64 5.5.8-arch1-1 \#1 SMP PREEMPT

In [ ]:
import os

strings = ['a','b','c','"b"','"c"','ä', 'cote', 'coté', 'côte', 'côté', 'и', 'й', 'і', 'ї']
locales_win = ['en_US', 'fr_FR', 'fr_CA', 'ru_RU']
locales_unix = [l + '.UTF-8' for l in locales_win]

locales = locales_win if os.name == 'nt' else locales_unix

Для UNIX-подобных ОС все локали придётся сгенерировать при помощи `locale-gen`

Тогда следующий пример будет правильно работать под Windows и UNIX

In [2]:
import icu

for l in locales_win:  # locales_win в том числе и под UNIX, иначе канадский французский не сработает...
    collator  = icu.Collator.createInstance(icu.Locale(l))
    print(l, sorted(strings, key=collator.getSortKey))

en_US ['"b"', '"c"', 'a', 'ä', 'b', 'c', 'cote', 'coté', 'côte', 'côté', 'и', 'і', 'ї', 'й']
fr_FR ['"b"', '"c"', 'a', 'ä', 'b', 'c', 'cote', 'coté', 'côte', 'côté', 'и', 'і', 'ї', 'й']
fr_CA ['"b"', '"c"', 'a', 'ä', 'b', 'c', 'cote', 'côte', 'coté', 'côté', 'и', 'і', 'ї', 'й']
ru_RU ['"b"', '"c"', 'и', 'і', 'ї', 'й', 'a', 'ä', 'b', 'c', 'cote', 'coté', 'côte', 'côté']


Следующий пример под Windows считает любой французский канадским, а под UNIX работает верно

In [3]:
import locale
import functools
import os
# https://docs.python.org/3/library/functools.html#functools.cmp_to_key

ol = locale.getlocale()

for l in locales:
    locale.setlocale(locale.LC_COLLATE, l)
    print(l, sorted(strings, key=functools.cmp_to_key(locale.strcoll)))

locale.setlocale(locale.LC_COLLATE, ol[0][:2] if os.name == 'nt' else '.'.join(ol))

en_US ['"b"', '"c"', 'a', 'ä', 'b', 'c', 'cote', 'coté', 'côte', 'côté', 'и', 'й', 'і', 'ї']
fr_FR ['"b"', '"c"', 'a', 'ä', 'b', 'c', 'cote', 'côte', 'coté', 'côté', 'и', 'й', 'і', 'ї']
fr_CA ['"b"', '"c"', 'a', 'ä', 'b', 'c', 'cote', 'côte', 'coté', 'côté', 'и', 'й', 'і', 'ї']
ru_RU ['"b"', '"c"', 'a', 'ä', 'b', 'c', 'cote', 'coté', 'côte', 'côté', 'и', 'і', 'ї', 'й']


'Ru'